<a href="https://colab.research.google.com/github/riccardocappi/Text-Adversarial-Attack/blob/adversarial-training/adversarial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
from builtins import len
!pip install textattack

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Imports

In [2]:
import textattack
import os
import transformers
from textattack.datasets import HuggingFaceDataset, Dataset
from textattack import Attacker
from textattack.attack_recipes import BAEGarg2019
from textattack.metrics.attack_metrics import (
    AttackQueries,
    AttackSuccessRate,
    WordsPerturbed,
)
import json
import os

2024-05-10 18:47:01.980039: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 18:47:02.959871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Helper methods

In [3]:
class SubsetHuggingFaceDataset(Dataset):
    def __init__(self, hfd, subset_size=10, offset=0, *args, **kwargs):
        super().__init__(dataset=hfd[offset:offset + subset_size], input_columns=hfd.input_columns,
                         label_names=hfd.label_names, *args, **kwargs)

    def __getitem__(self, i):
        """Return i-th sample."""
        if isinstance(i, int):
            return self._format_as_dict(self._dataset[i])

    def _format_as_dict(self, example):
        output = example[1]
        if self.label_map:
            output = self.label_map[output]
        if self.output_scale_factor:
            output = output / self.output_scale_factor

        return example[0], output

# Loading dataset and model
Getting train data in order to find attack transformation to add into the training set

In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Adversarial training BAEGarg2019

In [5]:
attack = BAEGarg2019.build(model_wrapper)
train_dataset = HuggingFaceDataset("imdb", split="train", shuffle=True)
eval_dataset = HuggingFaceDataset("imdb", split="test", shuffle=True)
subset_train = SubsetHuggingFaceDataset(train_dataset, subset_size=10)
subset_eval = SubsetHuggingFaceDataset(eval_dataset, subset_size=10)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split train.
textatt

In [6]:
training_args = textattack.TrainingArgs(
    num_epochs=3,
    num_clean_epochs=0,
    num_train_adv_examples=1,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    log_to_tb=True,
    parallel=True,
    random_seed=786
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    subset_train,
    subset_eval,
    training_args
)

## Train

In [7]:
trainer.train()

textattack: Writing logs to ./outputs/2024-05-10-18-47-29-399727/train_log.txt.
textattack: Wrote original training args to ./outputs/2024-05-10-18-47-29-399727/training_args.json.
/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 10
textattack:   Num epochs = 3
textattack:   Num clean epochs = 0
textattack:   Instantaneous batch size per device = 1
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 1
textattack:   Gradient accumulation steps = 1
textattack:   Total optimization steps = 33
textattack: ==========================================================
textattack: Epoch 1
textatt

Loss 0.00049: 100%|██████████| 11/11 [00:02<00:00,  4.77it/s]
textattack: Train accuracy: 100.00%
textattack: Eval accuracy: 90.00%
textattack: Best score found. Saved model to ./outputs/2024-05-10-18-47-29-399727/best_model/
textattack: ==========================================================
textattack: Epoch 2
textattack: Attacking model to generate new adversarial training set...
2024-05-10 18:48:08.149276: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 18:48:09.046205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  0%|          | 0/1 [00:00<?, ?it/s]2024-05-10 18:48:12.515699: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had n

ZeroDivisionError: division by zero